<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Win-funcs-begin" data-toc-modified-id="Win-funcs-begin-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Win funcs begin</a></span></li><li><span><a href="#Process-window" data-toc-modified-id="Process-window-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Process window</a></span></li><li><span><a href="#Call-func" data-toc-modified-id="Call-func-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Call func</a></span></li><li><span><a href="#Merge-outputs" data-toc-modified-id="Merge-outputs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merge outputs</a></span></li><li><span><a href="#Compare-vcf-file-sizes-(cluster-vs-source-ftp-server)" data-toc-modified-id="Compare-vcf-file-sizes-(cluster-vs-source-ftp-server)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compare vcf file sizes (cluster vs source ftp server)</a></span></li></ul></div>

In [9]:
# validate gz format
import gzip
pathgz = r"C:\Data\HUJI\hgdp\classes\windows\maf_0.49\count_dist_window_0.tsv.gz"
path = r"C:\Data\HUJI\hgdp\classes\windows\maf_0.49\count_dist_window_0.tsv"
with gzip.open(pathgz, 'rb') as f:
    file_contentgz = f.read()

with open(path, 'r') as f:
    file_content = f.read()



In [20]:
file_contentgzs = file_contentgz.decode()
for i in range(len(file_content)):
    if file_content[i]!=file_contentgzs[i]:
        print(i, file_content[i],file_contentgzs[i])
        break

In [68]:
# todo - think about the infra to get to each file, and naming conventions

# Process 012 files

In [69]:
#imports and consts
import os.path
import logging
import pandas as pd
import math

SUFFIX_POS = '.012.pos'
SUFFIX_012 = '.012'
OUTPUT_PATTERN_DIST_FILE = 'count_dist_slice_{slice_index}.tsv'
OUTPUT_PATTERN_SITES_FILE = 'sites_ids_slice_{slice_index}.tsv'
OUTPUT_PATTERN_LOG_FILE = 'Process012_{chr_name}_MAF_{maf}.log'

logger = logging.getLogger(__name__)

def _set_logger(chr_name, maf):
    log_file = OUTPUT_PATTERN_LOG_FILE.format(chr_name=chr_name, maf=maf)
    fhandler = logging.FileHandler(filename=log_file, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    # TODO - file is empty!
    logger.addHandler(fhandler)

def _log(msg, level=logging.INFO):
    print(msg)
    logger.log(level, msg)

In [39]:
def _validate_file_exist(p):
    if not os.path.isfile(p):
        print(f'Cant file file {p}')
        
def _get_chr_name(path_pos):
    with open(path_pos, 'r') as f:
        l = f.readline()
        char_name = l.split('\t')[0]
        return char_name
    
def _prepare_output_folder(base_output_path, chr_name):
    output_dir = f'{base_output_path}/{chr_name}/'
    print(f'output_dir is {output_dir}')
    os.makedirs(output_dir, exist_ok=True)
    return output_dir


def _build_pairwise_db(number, value):
    return [[value]*(number-i) for i in range (1,number)]

def _slice_calc_pairwise_distances_with_guardrails(slice_index, num_of_slices, slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected):
    # for each column, we calc the pairwise distances and add it to the grand total
    # for performance, we use 2 lists of lists, one for distances and one for counts
    window_pairwise_counts = _build_pairwise_db(len(slice_df), 0)
    window_pairwise_dist = _build_pairwise_db(len(slice_df), 0.0)
    for site_index in range(len(slice_df.columns)):
        _site_calc_pairwise_distances_with_guardrails(slice_index, num_of_slices, site_index, slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, window_pairwise_counts, window_pairwise_dist)
    return window_pairwise_counts, window_pairwise_dist


def _site_calc_pairwise_distances(slice_index, num_of_slices, site_index, genotypes, num_individuals, ref_freq, non_ref_freq, window_pairwise_counts, window_pairwise_dist):
    for i1 in range(num_individuals-1):
        i1_val = genotypes[i1]
        # if this entry is not valid for i1, no need to go over all the others, nothing to add to freq nor counts
        if i1%100==0:
            print(f'Slice: {slice_index}/{num_of_slices}. Site: {site_index}. Done with individual {i1}/{len(genotypes)}')
        if i1_val == -1:
            continue
        for i2 in range(i1+1, num_individuals):
            i2_val = genotypes[i2]
            if i2_val == -1:
                continue
            else:            
                # this is a valid entry, we add 1 to the count
                window_pairwise_counts[i1][i2-i1-1] += 1
                window_pairwise_dist[i1][i2-i1-1] += _calc_dist(i1_val, i2_val, ref_freq, non_ref_freq)

def _site_calc_pairwise_distances_with_guardrails(slice_index, num_of_slices, site_index, slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, window_pairwise_counts, window_pairwise_dist):
    genotypes = slice_df.iloc[:,site_index].values
    # get counts
    num_individuals = len(genotypes)
    # count the amount of not -1 in alleles
    num_valid_genotypes = len(genotypes[genotypes!=-1])
    non_ref_count = sum(genotypes[genotypes>0])
    ref_count = 2*num_valid_genotypes-non_ref_count
    non_ref_freq = float(non_ref_count)/(2*num_valid_genotypes)
    ref_freq = float(ref_count)/(2*num_valid_genotypes)
    print(f'Site index: {site_index}, non ref allele frequency: {non_ref_freq}')
    print(f'Site index: {site_index}, ref allele frequency: {ref_freq}')
    # guardrails 
    assert abs(ref_freq+non_ref_freq-1)<1e-04
    _check_guardrails(num_individuals, num_valid_genotypes, ref_count, non_ref_count, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)

    # now when we have all freq and counts, we can start the pairwise comparison
    # takes 0.5 seconds for one site. This means ~4 days if we are not seperating the task (which we will)
    # 178 sites took arounc 2 minutes.
    # Note that if we split this by chromozome, the big ones may have around 1000 in a given freq range, 
    # which can take ~8 minutes. 10k will be around 83 minutes - also cool.
    _site_calc_pairwise_distances(slice_index, num_of_slices, site_index, genotypes, num_individuals, ref_freq, non_ref_freq, window_pairwise_counts, window_pairwise_dist)

    
def _check_guardrails(num_individuals, num_valid_genotypes, ref_count, non_ref_count, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected):
    print(f'Check guardrails')
    # guardrail #1 - min_valid_sites_precentage
    percentage_valid_sites = float(num_valid_genotypes)/num_individuals
    print(f'Precentage of valid sites: {percentage_valid_sites}')
    if percentage_valid_sites < min_valid_sites_precentage:
        print(f'ERROR: % of valid sites is {percentage_valid_sites}, lower than allowd: {min_valid_sites_precentage}.')
        assert percentage_valid_sites < min_valid_sites_precentage

    # guardrail #2 mac/maf validation
    if (min_minor_freq_expected==-1 or max_minor_freq_expected==-1) and (min_minor_count_expected==-1 or max_minor_count_expected==-1):
        print(f'ERROR: min_minor_freq_expected, max_minor_freq_expected or min_minor_count_expected, max_minor_count_expected must be >-1')
        assert (min_minor_freq_expected==-1 or max_minor_freq_expected==-1) and (min_minor_count_expected==-1 or max_minor_count_expected==-1)
    
    #if maf: validate min_minor_freq_expected and max_minor_freq_expected
    if min_minor_freq_expected>-1 and max_minor_freq_expected>-1:
        minor_count = min(ref_count, non_ref_count)
        minor_freq = float(minor_count)/(2*num_valid_genotypes)
        print(f'Minor allele frequency: {minor_freq}')
        if minor_freq < min_minor_freq_expected:
            print(f'ERROR: minor frequency is too low - {minor_freq}, allowd: {min_minor_freq_expected}.')
            assert minor_freq < min_minor_freq_expected
        if minor_freq > max_minor_freq_expected:
            print(f'ERROR: minor frequency is too high - {minor_freq}, allowd: {max_minor_freq_expected}.')
            assert minor_freq > max_minor_freq_expected
    
    #if mac: validate min_minor_count_expected and max_minor_count_expected
    if min_minor_count_expected>-1 and max_minor_count_expected>-1:
        minor_count = min(ref_count, non_ref_count)
        print(f'Minor allele count: {minor_count}')
        if minor_count < min_minor_count_expected:
            print(f'ERROR: minor frequency is too low - {minor_freq}, allowd: {min_minor_freq_expected}.')
            assert minor_count < min_minor_count_expected
        if minor_count > max_minor_count_expected:
            print(f'ERROR: minor frequency is too high - {minor_freq}, allowd: {max_minor_freq_expected}.')
            assert minor_count > max_minor_count_expected
    print(f'Passed guardrails')
    
    
def _write_output_sites_file(output_sites_file, slice_df):
    # TODO - output per site the calculated number of missing values and MAF
    sites_ids = slice_df.columns.values
    with open(output_sites_file, 'w') as f:
        for site_id in sites_ids:
            f.write(site_id.replace('_','\t') + '\n')


In [67]:
def _calc_dist(i1_val, i2_val, ref_freq, non_ref_freq):
    #print(i1_val, i2_val, ref_freq, non_ref_freq)
    # from VCFtools manual:
    # "Genotypes are represented as 0, 1 and 2, where the number represent that number of non-reference alleles"
    # So - v1_val and v2_val are the amount of ***non-ref alleles***.
    # The distance function is:
    # 1/4[(1-f_a)(Iac+Iad) + (1-f_b)(Ibc+Ibd)]
    # Now, for each combination of v1_val and v2_val, we can compute the distance.
        
    # 0,0 - v1_val=0 and v2_val=0:
    #    1/4[(1-f_ref)(1+1) + (1-f_ref)(1+1)] = 1/4[(1-f_ref)(4)] = (1-f_ref)
    
    # 0,1 - v1_val=0 and v2_val=1:
    #    1/4[(1-f_ref)(1+0) + (1-f_ref)(1+0)] = 1/4[(1-f_ref)(2)] = 1/2(1-f_ref)
    
    # 0,2 - v1_val=0 and v2_val=2:
    #    1/4[(1-f_ref)(0+0) + (1-f_ref)(0+0)] = 0
    
    # 1,1 - v1_val=1 and v2_val=1:
    #    1/4[(1-f_non_ref)(1+0) + (1-f_ref)(1+0)] = 1/4[(1-f_non_ref)+(1-f_ref)]
    # note: if f_non_ref+f_ref=1, we get 1/4
    
    # 1,2 - v1_val=1 and v2_val=2:
    #    1/4[(1-f_non_ref)(1+1) + (1-f_ref)(0+0)] = 1/4[(1-f_non_ref)(2)] = 1/2(1-f_non_ref)
    
    # 2,2 - v1_val=2 and v2_val=2:
    #    1/4[(1-f_non_ref)(1+1) + (1-f_non_ref)(1+1)] = 1/4[(1-f_non_ref)(4)] = (1-f_non_ref)
    
    # Also, note that it is symetric:

    # 1,0 - v1_val=1 and v2_val=0:
    #    1/4[(1-f_ref)(1+1) + (1-f_non_ref)(0+0)] = 1/4[(1-f_ref)(2)] = 1/2(1-f_ref)
    
    # 2,1 - v1_val=2 and v2_val=1:
    #    1/4[(1-f_non_ref)(1+0) + (1-f_non_ref)(0+1)] = 1/4[(1-f_non_ref)(2)] = 1/2(1-f_non_ref)

    # 2,0 - v1_val=2 and v2_val=0:
    #    1/4[(1-f_non_ref)(0+0) + (1-f_non_ref)(0+0)] = 0
    
    # formula to use: 
    #   (1-f_non_ref)(v1_val*v2_val) + (1-f_ref)((2-v1_val)*(2-v2_val))/4
    return ((1-non_ref_freq)*(i1_val*i2_val) + (1-ref_freq)*((2-i1_val)*(2-i2_val)))/4

In [68]:
def _calc_dist_directly(i1_val, i2_val, ref_freq, non_ref_freq):
    # set a,b
    # both ref
    if i1_val == 0:
        a='ref'
        b='ref'
    if i1_val == 1:
        a='non_ref'
        b='ref'
    # both non ref
    if i1_val == 2:
        a='non_ref'
        b='non_ref'
    # set c,d
    if i2_val == 0:
        c='ref'
        d='ref'
    if i2_val == 1:
        c='non_ref'
        d='ref'
    if i2_val == 2:
        c='non_ref'
        d='non_ref'
    Iac = 1 if a==c else 0
    Iad = 1 if a==d else 0
    Ibc = 1 if b==c else 0
    Ibd = 1 if b==d else 0
    f_a = ref_freq if a=='ref' else non_ref_freq
    f_b = ref_freq if b=='ref' else non_ref_freq
    return (((1-f_a)*(Iac+Iad)) + ((1-f_b)*(Ibc+Ibd)))/4.0

In [75]:
for i in range(10000000):
    _calc_dist(0,2,f_ref,f_non_ref)

In [76]:
for i in range(10000000):
    _calc_dist_directly(0,2,f_ref,f_non_ref)

In [66]:
# validate _calc_dist
f_ref = 0.1
f_non_ref = 0.8
# assert symatry
assert _calc_dist(1,0,f_ref,f_non_ref) == _calc_dist(0,1,f_ref,f_non_ref)
assert _calc_dist(2,0,f_ref,f_non_ref) == _calc_dist(0,2,f_ref,f_non_ref)
assert _calc_dist(2,1,f_ref,f_non_ref) == _calc_dist(1,2,f_ref,f_non_ref)
# assert calc
assert _calc_dist(0,0,f_ref,f_non_ref) == _calc_dist_directly(0,0,f_ref,f_non_ref)
assert _calc_dist(0,1,f_ref,f_non_ref) == _calc_dist_directly(0,1,f_ref,f_non_ref)
assert _calc_dist(0,2,f_ref,f_non_ref) == _calc_dist_directly(0,2,f_ref,f_non_ref)
assert _calc_dist(1,2,f_ref,f_non_ref) == _calc_dist_directly(1,2,f_ref,f_non_ref)
assert _calc_dist(2,2,f_ref,f_non_ref) == _calc_dist_directly(2,2,f_ref,f_non_ref)



1 0 0.1 0.8
0 1 0.1 0.8
2 0 0.1 0.8
0 2 0.1 0.8
2 1 0.1 0.8
1 2 0.1 0.8
0 0 0.1 0.8
0 1 0.1 0.8
0 2 0.1 0.8
1 2 0.1 0.8
2 2 0.1 0.8


In [99]:
#-1 1 2 2
#--rnnnnn
nr = 5/6
r=1/6
r

0.16666666666666666

In [102]:
_calc_dist(1,2,r,nr)

0.08333333333333331

In [85]:
expected_distances_per_colus = {
    # locus 1
    '1':{
        '1':{
            '2':0.25,
            '3':0.375,
            '4':0.375,
        },
        '2':{
            '3':0.375,
            '4':0.375,
        },
        '3':{
            '4':0.75,
        }
    },
    # locus 2
    '2':{
        '1':{
            '2':0.75,
            '3':0.375,
            '4':0.375,
        },
        '2':{
            '3':0.375,
            '4':0.375,
        },
        '3':{
            '4':0.25,
        }
    },
    # locus 3
    '3':{
        '1':{
            '2':0.5,
            '3':0.0,
            '4':0.0,
        },
        '2':{
            '3':0.0,
            '4':0.0,
        },
        '3':{
            '4':0.5,
        }
    },
    # locus 4
    '4':{
        '2':{
            '3':0.41667,
            '4':0.41667,
        },
        '3':{
            '4':0.83334,
        }
    }
}

In [87]:
expected_distances_per_colus

{'1': {'1': {'2': 0.25, '3': 0.375, '4': 0.375},
  '2': {'3': 0.375, '4': 0.375},
  '3': {'4': 0.75}},
 '2': {'1': {'2': 0.75, '3': 0.375, '4': 0.375},
  '2': {'3': 0.375, '4': 0.375},
  '3': {'4': 0.25}},
 '3': {'1': {'2': 0.5, '3': 0.0, '4': 0.0},
  '2': {'3': 0.0, '4': 0.0},
  '3': {'4': 0.5}},
 '4': {'2': {'3': 0.41667, '4': 0.41667}, '3': {'4': 0.83334}}}

In [54]:
def _test(v1_val, v2_val):
    print(v1_val, v2_val, f'(1-f_a){(v1_val*v2_val)} + (1-f_r){(2-v1_val)*(2-v2_val)}')
for v1 in range(3):
    for v2 in range(v1,3):
        _test(v1, v2)

0 0 (1-f_a)0 + (1-f_r)4
0 1 (1-f_a)0 + (1-f_r)2
0 2 (1-f_a)0 + (1-f_r)0
1 1 (1-f_a)1 + (1-f_r)1
1 2 (1-f_a)2 + (1-f_r)0
2 2 (1-f_a)4 + (1-f_r)0


## Win funcs begin

In [10]:
import pandas as pd
def get_window(classes_folder, mac_maf, class_name, window_index):
    class_012_path_template = classes_folder + "chr{chr_id}\{class_name}.012"
    windows_indexes_files_folder = classes_folder + r"windows_indexes/"
    windows_indexes_path_template = windows_indexes_files_folder + "windows_indexes_for_class_{class_name}.json"

    print(f'Class: {mac_maf}_{class_name}, window index: {window_index}')
    # read indexes of window
    windows_indexes_path = windows_indexes_path_template.format(class_name=class_name)
    with open(windows_indexes_path) as f:
        windows_indexes = json.load(f)
    window_indexes = windows_indexes[window_index]
    print(f'There are {len(window_indexes)} indexes in window indexes list')
    chr_id2indexes = dict()
    for char_id_index in window_indexes:
        chr_id, index = char_id_index.split(';',1)
        # if chr_id not in the dict, we get an empty list
        indexes = chr_id2indexes.get(chr_id, [])
        indexes.append(int(index))
        chr_id2indexes[chr_id] = indexes
    #sort the indexes so that we can easily read the file
    {k:v.sort() for k, v in chr_id2indexes.items()}

    # go over the chr_ids, and get from the file the correct columns
    class_012_df = pd.DataFrame()
    for chr_id, indexes in chr_id2indexes.items():
        class_012_path = class_012_path_template.format(chr_id = chr_id, mac_maf = mac_maf, class_name = class_name)
        # get number of columns in chr:
        with open(class_012_path,'r') as f:
            num_columns = len(f.readline().split('\t'))
        print(f'{len(indexes)} / {num_columns-1} sites will be used from file {class_012_path}')
        names = [f'chr{chr_id}_idx{i}' for i in range(num_columns)]
        # we add one as the csv contains the individual is in the first index
        cols_to_uset = [i+1 for i in indexes]
        # read file to pandas df
        selected_indexes_012_df = pd.read_csv(class_012_path, sep='\t', names= names, usecols = cols_to_uset)
        if class_012_df.empty:
            class_012_df = selected_indexes_012_df
        else:
            class_012_df = class_012_df.join(selected_indexes_012_df)
    # validate the number of columns is the number of window indexes
    assert class_012_df.shape[1] == len(window_indexes)
    print(f'{class_012_df.shape[1]} sites in window')
    return class_012_df




In [2]:
def window_calc_pairwise_distances_with_guardrails(window_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected):
    # for each column, we calc the pairwise distances and add it to the grand total
    # for performance, we use 2 lists of lists, one for distances and one for counts
    window_pairwise_counts = _build_pairwise_db(len(window_df), 0)
    window_pairwise_dist = _build_pairwise_db(len(window_df), 0.0)
    for site_index in range(len(window_df.columns)):
        site_calc_pairwise_distances_with_guardrails(window_df, site_index, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected, window_pairwise_counts, window_pairwise_dist)
    return window_pairwise_counts, window_pairwise_dist


In [16]:
def site_calc_pairwise_distances_with_guardrails(window_df, site_index, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected, window_pairwise_counts, window_pairwise_dist):
    genotypes = window_df.iloc[:,site_index].values
    print('genotypes', genotypes)
    # get counts
    num_individuals = len(genotypes)
    print('num_individuals', num_individuals)
    # count the amount of not -1 in alleles
    num_valid_genotypes = len(genotypes[genotypes!=-1])
    print('num_valid_genotypes', num_valid_genotypes)
    non_ref_count = sum(genotypes[genotypes>0])
    ref_count = 2*num_valid_genotypes-non_ref_count
    non_ref_freq = float(non_ref_count)/(2*num_valid_genotypes)
    ref_freq = float(ref_count)/(2*num_valid_genotypes)
    print(f'Site index: {site_index}, non ref allele frequency: {non_ref_freq}')
    print(f'Site index: {site_index}, ref allele frequency: {ref_freq}')
    # guardrails 
    assert abs(ref_freq+non_ref_freq-1)<1e-04
    _check_guardrails(num_individuals, num_valid_genotypes, ref_count, non_ref_count, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected)
    site_calc_pairwise_distances(genotypes, num_individuals, ref_freq, non_ref_freq, window_pairwise_counts, window_pairwise_dist)


In [24]:
def site_calc_pairwise_distances(genotypes, num_individuals, ref_freq, non_ref_freq, window_pairwise_counts, window_pairwise_dist):
    for i1 in range(num_individuals-1):
        i1_val = genotypes[i1]
        print('i1', i1, 'i1_val', i1_val)
        # if this entry is not valid for i1, no need to go over all the others, nothing to add to freq nor counts
        if i1%100==0:
            print(f'Done with individual {i1}/{len(genotypes)}')
        if i1_val == -1:
            continue
        for i2 in range(i1+1, num_individuals):
            i2_val = genotypes[i2]
            print('i2', i2, 'i2_val', i2_val)
            if i2_val == -1:
                continue
            else:            
                # this is a valid entry, we add 1 to the count
                window_pairwise_counts[i1][i2-i1-1] += 1
                print('dist',_calc_dist(i1_val, i2_val, ref_freq, non_ref_freq))
                window_pairwise_dist[i1][i2-i1-1] += _calc_dist(i1_val, i2_val, ref_freq, non_ref_freq)

In [18]:
# the output is in couples of <count>,<distance>
# the count is the number of valied sites on which the distances is calculated
def write_pairwise_distances(output_count_dist_file, window_pairwise_counts, window_pairwise_dist):
    with open(output_count_dist_file,'w') as f:
        for counts,dists in zip(window_pairwise_counts, window_pairwise_dist):
            s = ' '.join(f'{c}{round(d, 5)}' for c,d in zip(counts, dists)) + '\n'
            f.write(s)

## Process window

In [8]:
class_name='all'
# PARAMS
# UTILS FOR PARAMS
classes_folder = r"../mock/classes/"
class_012_path_template = classes_folder + "chr{chr_id}/{class_name}.012"
windows_indexes_files_folder = classes_folder + r"windows_indexes/"
windows_indexes_path_template = windows_indexes_files_folder + "windows_indexes_for_class_{class_name}.json"

# if we have less than this which are valid (not -1), site is not included in calc.
min_valid_sites_precentage = 0.1
# if not in range - will raise a warning
mac_maf = 'maf'
min_minor_freq_expected = 0.0
max_minor_freq_expected = 0.5
min_minor_count_expected = -1
max_minor_count_expected = -1
class_name = 'all'

windows_indexes_path = windows_indexes_path_template.format(class_name=class_name)
output_dir = f'{classes_folder}windows/{class_name}/'

In [45]:
window_df = get_window(classes_folder, mac_maf, class_name, window_index=0)
print(window_df)
window_pairwise_counts, window_pairwise_dist = window_calc_pairwise_distances_with_guardrails(window_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected)
window_pairwise_counts, window_pairwise_dist

Class: maf_all, window index: 0
There are 1 indexes in window indexes list
1 / 4 sites will be used from file ../mock/classes/chr1\all.012
1 sites in window
   chr1_idx1
0          1
1          1
2          0
3          0
Site index: 0, non ref allele frequency: 0.25
Site index: 0, ref allele frequency: 0.75
Check guardrails
Precentage of valid sites: 1.0
Minor allele frequency: 0.25
Passed guardrails
i1 0 i1_val 1
Done with individual 0/4
i2 1 i2_val 1
1 1 0.75 0.25
dist 0.25
1 1 0.75 0.25
i2 2 i2_val 0
1 0 0.75 0.25
dist 0.125
1 0 0.75 0.25
i2 3 i2_val 0
1 0 0.75 0.25
dist 0.125
1 0 0.75 0.25
i1 1 i1_val 1
i2 2 i2_val 0
1 0 0.75 0.25
dist 0.125
1 0 0.75 0.25
i2 3 i2_val 0
1 0 0.75 0.25
dist 0.125
1 0 0.75 0.25
i1 2 i1_val 0
i2 3 i2_val 0
0 0 0.75 0.25
dist 0.25
0 0 0.75 0.25


([[1, 1, 1], [1, 1], [1]], [[0.25, 0.125, 0.125], [0.125, 0.125], [0.25]])

In [29]:
window_df = get_window(classes_folder, mac_maf, class_name, window_index=0)
print(window_df)
window_pairwise_counts, window_pairwise_dist = window_calc_pairwise_distances_with_guardrails(window_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected)
window_pairwise_counts, window_pairwise_dist

Class: maf_all, window index: 0
There are 1 indexes in window indexes list
1 / 4 sites will be used from file ../mock/classes/chr1\all.012
1 sites in window
   chr1_idx1
0          1
1          1
2          0
3          0
Site index: 0, non ref allele frequency: 0.25
Site index: 0, ref allele frequency: 0.75
Check guardrails
Precentage of valid sites: 1.0
Minor allele frequency: 0.25
Passed guardrails
i1 0 i1_val 1
Done with individual 0/4
i2 1 i2_val 1
dist 0.8125
i2 2 i2_val 0
dist 0.125
i2 3 i2_val 0
dist 0.125
i1 1 i1_val 1
i2 2 i2_val 0
dist 0.125
i2 3 i2_val 0
dist 0.125
i1 2 i1_val 0
i2 3 i2_val 0
dist 0.25


([[1, 1, 1], [1, 1], [1]], [[0.8125, 0.125, 0.125], [0.125, 0.125], [0.25]])

In [71]:
# PARAMS
# if we have less than this which are valid (not -1), site is not included in calc.
min_valid_sites_precentage = 0.1

# if not in range - will raise a warning
min_minor_freq_expected = 0.49
max_minor_freq_expected = 0.5
min_minor_count_expected = -1
max_minor_count_expected = -1

mac_maf = 'maf'
class_name = '0.49'
window_index = 0
classes_folder = r"C:\Data\HUJI\hgdp\classes/"

# FUNCTION
OUTPUT_PATTERN_DIST_FILE = 'count_dist_window_{window_index}.tsv'
output_dir = f'{classes_folder}windows/{mac_maf}_{class_name}/'
os.makedirs(output_dir, exist_ok=True)
output_count_dist_file = output_dir + OUTPUT_PATTERN_DIST_FILE.format(window_index=window_index)
       
window_df = get_window(classes_folder, mac_maf, class_name, window_index)

# 100 indexes takes ~5 minutes 
# using percision of 5 decimals we generates a file of ~5MB
# we have 322,483 windows of 100
# will take over 3 years to process on one machine
# output is about 12 TB
window_pairwise_counts, window_pairwise_dist = window_calc_pairwise_distances_with_guardrails(window_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)


Site index: 0, non ref allele frequency: 0.4924406047516199
Site index: 0, ref allele frequency: 0.5075593952483801
Check guardrails
Precentage of valid sites: 0.9967707212055974
Minor allele frequency: 0.4924406047516199
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 1, non ref allele frequency: 0.4924078091106291
Site index: 1, ref allele frequency: 0.5075921908893709
Check guardrails
Precentage of valid sites: 0.9924650161463939
Minor allele frequency: 0.4924078091106291
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Do

Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 16, non ref allele frequency: 0.4989212513484358
Site index: 16, ref allele frequency: 0.5010787486515642
Check guardrails
Precentage of valid sites: 0.9978471474703983
Minor allele frequency: 0.4989212513484358
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 17, non ref allele frequency: 0.5097402597402597
Site index: 17, ref allele frequency: 0.4902597402597403
Check guardrails
Precentage of valid sites: 0.9946178686759957
Minor allele frequency: 0.49

Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 32, non ref allele frequency: 0.4964788732394366
Site index: 32, ref allele frequency: 0.5035211267605634
Check guardrails
Precentage of valid sites: 0.9171151776103337
Minor allele frequency: 0.4964788732394366
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 33, non ref allele frequency: 0.5
Site index: 33, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.9924650161463939
Minor allele frequency: 0.5
Passed guardrails
Done with in

Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 48, non ref allele frequency: 0.4967637540453074
Site index: 48, ref allele frequency: 0.5032362459546925
Check guardrails
Precentage of valid sites: 0.9978471474703983
Minor allele frequency: 0.4967637540453074
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 49, non ref allele frequency: 0.5005813953488372
Site index: 49, ref allele frequency: 0.4994186046511628
Check guardrails
Precentage of valid sites: 0.9257265877287406
Minor allele frequency: 0.49

Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 64, non ref allele frequency: 0.5
Site index: 64, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 1.0
Minor allele frequency: 0.5
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 65, non ref allele frequency: 0.4951403887688985
Site index: 65, ref allele frequency: 0.5048596112311015
Check guardrails
Precentage of valid sites: 0.9967707212055974
Minor allele frequency: 0.4951403887688985
Passed guardrails
Done with individual 0/929


Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 80, non ref allele frequency: 0.49083063646170444
Site index: 80, ref allele frequency: 0.5091693635382956
Check guardrails
Precentage of valid sites: 0.9978471474703983
Minor allele frequency: 0.49083063646170444
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 81, non ref allele frequency: 0.4956521739130435
Site index: 81, ref allele frequency: 0.5043478260869565
Check guardrails
Precentage of valid sites: 0.9903121636167922
Minor allele frequency: 0.

Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 96, non ref allele frequency: 0.49457111834962
Site index: 96, ref allele frequency: 0.50542888165038
Check guardrails
Precentage of valid sites: 0.9913885898815931
Minor allele frequency: 0.49457111834962
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 97, non ref allele frequency: 0.5048543689320388
Site index: 97, ref allele frequency: 0.49514563106796117
Check guardrails
Precentage of valid sites: 0.9978471474703983
Minor allele frequency: 0.4951456

In [98]:
dummy_df = window_df[window_df.columns[:4]]
window_pairwise_counts, window_pairwise_dist = window_calc_pairwise_distances_with_guardrails(dummy_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected, min_minor_count_expected, max_minor_count_expected)


Site index: 0, non ref allele frequency: 0.4924406047516199
Site index: 0, ref allele frequency: 0.5075593952483801
Check guardrails
Precentage of valid sites: 0.9967707212055974
Minor allele frequency: 0.4924406047516199
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Done with individual 800/929
Done with individual 900/929
Site index: 1, non ref allele frequency: 0.4924078091106291
Site index: 1, ref allele frequency: 0.5075921908893709
Check guardrails
Precentage of valid sites: 0.9924650161463939
Minor allele frequency: 0.4924078091106291
Passed guardrails
Done with individual 0/929
Done with individual 100/929
Done with individual 200/929
Done with individual 300/929
Done with individual 400/929
Done with individual 500/929
Done with individual 600/929
Done with individual 700/929
Do

In [85]:
write_pairwise_distances(output_count_dist_file, window_pairwise_counts, window_pairwise_dist)

In [8]:

def build_per_slice_distances_matrix(
    path_to_012_files, 
    base_output_path, 
    desired_num_of_sites_per_output,
    min_minor_freq_expected, 
    max_minor_freq_expected, 
    min_valid_sites_precentage):
    # outputs:
    # for each group i in 1,...,k:
    # 1. count_dist : space seperated pairwise distances and count of valid sites.
    #    This is the sum over the valid sites used, and the format in each entry is <sum>,<l'>.
    # dummy example for 3 individuals, over l=5, where individual number 3 has 2 invalid entries (-1 -1 1 0 2)
    # 0.9,5 0.7,3
    # 0.8,3 
    # the ',3' indicates that there are only 3 valied sited for individuals 1 and 3 and 2 and 3
    # 2. {i}_sites_ids : <site_id> <number of non -1 entris>


    # Validate inputs
    path_012 = f'{path_to_012_files}{SUFFIX_012}'
    path_pos = f'{path_to_012_files}{SUFFIX_POS}'
    chr_name = _get_chr_name(path_pos)

    _set_logger(chr_name, min_minor_freq_expected)

    _validate_file_exist(path_012)
    _validate_file_exist(path_pos)

    # prepare output folder
    _log(f'chr_name is {chr_name}')
    output_dir = _prepare_output_folder(base_output_path, chr_name)

    # load the data
    # get sites positions to use as column names
    with open(path_pos, 'r') as f:
        sites_pos = [l.replace('\t','_').replace('\n','') for l in f.readlines()]
    all_df = pd.read_csv(path_012, sep='\t', names = ['ind_id'] + sites_pos)
    # drop first column - individual ids
    all_df = all_df.drop(columns=['ind_id'])
    num_sites = len(all_df.columns)
    num_of_slices = int(float(num_sites)/desired_num_of_sites_per_output)
    size_of_last_slice = num_sites % desired_num_of_sites_per_output
    _log(f'Total num of sites: {num_sites}')
    _log(f'Desired size of slice: {desired_num_of_sites_per_output}')
    _log(f'Number of slices: {num_of_slices+1}')
    _log(f'Size of last slice: {size_of_last_slice}')

    for slice_index in range(num_of_slices+1):
        _log(f'{chr_name}: slice {slice_index}/{num_of_slices}')

        min_col_index = slice_index*desired_num_of_sites_per_output
        # in case we have less than desired_num_of_sites_per_output, pandas will only take those we have, nice!
        max_col_index = min_col_index + desired_num_of_sites_per_output
        slice_df = all_df.iloc[:,min_col_index:max_col_index]
        num_of_sites = len(slice_df.columns)
        _log(f'\n#############\n')
        _log(f'Slice index: {slice_index}. Slice size: {num_of_sites} ')
        output_count_dist_file = output_dir + OUTPUT_PATTERN_DIST_FILE.format(slice_index=slice_index)
        output_sites_file = output_dir + OUTPUT_PATTERN_SITES_FILE.format(slice_index=slice_index)
        # todo - maybe write to this file also the number of -1, 0, 1 and 2?
        _write_output_sites_file(output_sites_file, slice_df)
        _log(f'Output files: {output_count_dist_file} {output_sites_file}')
        window_pairwise_counts, window_pairwise_dist = _slice_calc_pairwise_distances_with_guardrails(slice_index, num_of_slices, slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)
        _write_pairwise_distances(output_count_dist_file, window_pairwise_counts, window_pairwise_dist)



## Call func

In [10]:
path_to_012_files = 'C:\Data\HUJI\hgdp\mock\out'
base_output_path = 'C:\Data\HUJI\hgdp\mock'
build_per_slice_distances_matrix(path_to_012_files, base_output_path, desired_num_of_sites_per_output=100, min_minor_freq_expected=0.49, max_minor_freq_expected=0.5, min_valid_sites_precentage=0.1)
build_per_slice_distances_matrix(path_to_012_files, base_output_path, desired_num_of_sites_per_output=50, min_minor_freq_expected=0.49, max_minor_freq_expected=0.5, min_valid_sites_precentage=0.1)

chr_name is chr9
output_dir is C:\Data\HUJI\hgdp\mock/chr9/
Total num of sites: 178
Desired size of slice: 100
Number of slices: 2
Size of last slice: 78
chr9: slice 0/1

#############

Slice index: 0. Slice size: 100 
Output files: C:\Data\HUJI\hgdp\mock/chr9/count_dist_slice_0.tsv C:\Data\HUJI\hgdp\mock/chr9/sites_ids_slice_0.tsv
Site index: 0, non ref allele frequency: 0.5
Site index: 0, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.4951560818083961
Minor allele frequency: 0.5
Passed guardrails
Slice: 0/1. Site: 0. Done with individual 0/929
Slice: 0/1. Site: 0. Done with individual 100/929
Slice: 0/1. Site: 0. Done with individual 200/929
Slice: 0/1. Site: 0. Done with individual 300/929
Slice: 0/1. Site: 0. Done with individual 400/929
Slice: 0/1. Site: 0. Done with individual 500/929
Slice: 0/1. Site: 0. Done with individual 600/929
Slice: 0/1. Site: 0. Done with individual 700/929
Slice: 0/1. Site: 0. Done with individual 800/929
Slice: 0/1. Site: 0. Do

Slice: 0/1. Site: 11. Done with individual 100/929
Slice: 0/1. Site: 11. Done with individual 200/929
Slice: 0/1. Site: 11. Done with individual 300/929
Slice: 0/1. Site: 11. Done with individual 400/929
Slice: 0/1. Site: 11. Done with individual 500/929
Slice: 0/1. Site: 11. Done with individual 600/929
Slice: 0/1. Site: 11. Done with individual 700/929
Slice: 0/1. Site: 11. Done with individual 800/929
Slice: 0/1. Site: 11. Done with individual 900/929
Site index: 12, non ref allele frequency: 0.4919786096256685
Site index: 12, ref allele frequency: 0.5080213903743316
Check guardrails
Precentage of valid sites: 0.20129171151776104
Minor allele frequency: 0.4919786096256685
Passed guardrails
Slice: 0/1. Site: 12. Done with individual 0/929
Slice: 0/1. Site: 12. Done with individual 100/929
Slice: 0/1. Site: 12. Done with individual 200/929
Slice: 0/1. Site: 12. Done with individual 300/929
Slice: 0/1. Site: 12. Done with individual 400/929
Slice: 0/1. Site: 12. Done with individual 50

Slice: 0/1. Site: 24. Done with individual 200/929
Slice: 0/1. Site: 24. Done with individual 300/929
Slice: 0/1. Site: 24. Done with individual 400/929
Slice: 0/1. Site: 24. Done with individual 500/929
Slice: 0/1. Site: 24. Done with individual 600/929
Slice: 0/1. Site: 24. Done with individual 700/929
Slice: 0/1. Site: 24. Done with individual 800/929
Slice: 0/1. Site: 24. Done with individual 900/929
Site index: 25, non ref allele frequency: 0.5
Site index: 25, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.8460710441334769
Minor allele frequency: 0.5
Passed guardrails
Slice: 0/1. Site: 25. Done with individual 0/929
Slice: 0/1. Site: 25. Done with individual 100/929
Slice: 0/1. Site: 25. Done with individual 200/929
Slice: 0/1. Site: 25. Done with individual 300/929
Slice: 0/1. Site: 25. Done with individual 400/929
Slice: 0/1. Site: 25. Done with individual 500/929
Slice: 0/1. Site: 25. Done with individual 600/929
Slice: 0/1. Site: 25. Done with individu

Slice: 0/1. Site: 35. Done with individual 600/929
Slice: 0/1. Site: 35. Done with individual 700/929
Slice: 0/1. Site: 35. Done with individual 800/929
Slice: 0/1. Site: 35. Done with individual 900/929
Site index: 36, non ref allele frequency: 0.5015948963317385
Site index: 36, ref allele frequency: 0.4984051036682616
Check guardrails
Precentage of valid sites: 0.6749192680301399
Minor allele frequency: 0.4984051036682616
Passed guardrails
Slice: 0/1. Site: 36. Done with individual 0/929
Slice: 0/1. Site: 36. Done with individual 100/929
Slice: 0/1. Site: 36. Done with individual 200/929
Slice: 0/1. Site: 36. Done with individual 300/929
Slice: 0/1. Site: 36. Done with individual 400/929
Slice: 0/1. Site: 36. Done with individual 500/929
Slice: 0/1. Site: 36. Done with individual 600/929
Slice: 0/1. Site: 36. Done with individual 700/929
Slice: 0/1. Site: 36. Done with individual 800/929
Slice: 0/1. Site: 36. Done with individual 900/929
Site index: 37, non ref allele frequency: 0.5


Slice: 0/1. Site: 47. Done with individual 100/929
Slice: 0/1. Site: 47. Done with individual 200/929
Slice: 0/1. Site: 47. Done with individual 300/929
Slice: 0/1. Site: 47. Done with individual 400/929
Slice: 0/1. Site: 47. Done with individual 500/929
Slice: 0/1. Site: 47. Done with individual 600/929
Slice: 0/1. Site: 47. Done with individual 700/929
Slice: 0/1. Site: 47. Done with individual 800/929
Slice: 0/1. Site: 47. Done with individual 900/929
Site index: 48, non ref allele frequency: 0.5
Site index: 48, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.620021528525296
Minor allele frequency: 0.5
Passed guardrails
Slice: 0/1. Site: 48. Done with individual 0/929
Slice: 0/1. Site: 48. Done with individual 100/929
Slice: 0/1. Site: 48. Done with individual 200/929
Slice: 0/1. Site: 48. Done with individual 300/929
Slice: 0/1. Site: 48. Done with individual 400/929
Slice: 0/1. Site: 48. Done with individual 500/929
Slice: 0/1. Site: 48. Done with individua

...

Slice: 1/3. Site: 34. Done with individual 100/929
Slice: 1/3. Site: 34. Done with individual 200/929
Slice: 1/3. Site: 34. Done with individual 300/929
Slice: 1/3. Site: 34. Done with individual 400/929
Slice: 1/3. Site: 34. Done with individual 500/929
Slice: 1/3. Site: 34. Done with individual 600/929
Slice: 1/3. Site: 34. Done with individual 700/929
Slice: 1/3. Site: 34. Done with individual 800/929
Slice: 1/3. Site: 34. Done with individual 900/929
Site index: 35, non ref allele frequency: 0.5
Site index: 35, ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.10118406889128095
Minor allele frequency: 0.5
Passed guardrails
Slice: 1/3. Site: 35. Done with individual 0/929
Slice: 1/3. Site: 35. Done with individual 100/929
Slice: 1/3. Site: 35. Done with individual 200/929
Slice: 1/3. Site: 35. Done with individual 300/929
Slice: 1/3. Site: 35. Done with individual 400/929
Slice: 1/3. Site: 35. Done with individual 500/929
Slice: 1/3. Site: 35. Done with individ

Slice: 1/3. Site: 45. Done with individual 200/929
Slice: 1/3. Site: 45. Done with individual 300/929
Slice: 1/3. Site: 45. Done with individual 400/929
Slice: 1/3. Site: 45. Done with individual 500/929
Slice: 1/3. Site: 45. Done with individual 600/929
Slice: 1/3. Site: 45. Done with individual 700/929
Slice: 1/3. Site: 45. Done with individual 800/929
Slice: 1/3. Site: 45. Done with individual 900/929
Site index: 46, non ref allele frequency: 0.5049019607843137
Site index: 46, ref allele frequency: 0.4950980392156863
Check guardrails
Precentage of valid sites: 0.21959095801937567
Minor allele frequency: 0.4950980392156863
Passed guardrails
Slice: 1/3. Site: 46. Done with individual 0/929
Slice: 1/3. Site: 46. Done with individual 100/929
Slice: 1/3. Site: 46. Done with individual 200/929
Slice: 1/3. Site: 46. Done with individual 300/929
Slice: 1/3. Site: 46. Done with individual 400/929
Slice: 1/3. Site: 46. Done with individual 500/929
Slice: 1/3. Site: 46. Done with individual 60

Slice: 2/3. Site: 6. Done with individual 100/929
Slice: 2/3. Site: 6. Done with individual 200/929
Slice: 2/3. Site: 6. Done with individual 300/929
Slice: 2/3. Site: 6. Done with individual 400/929
Slice: 2/3. Site: 6. Done with individual 500/929
Slice: 2/3. Site: 6. Done with individual 600/929
Slice: 2/3. Site: 6. Done with individual 700/929
Slice: 2/3. Site: 6. Done with individual 800/929
Slice: 2/3. Site: 6. Done with individual 900/929
Site index: 7, non ref allele frequency: 0.5087579617834395
Site index: 7, ref allele frequency: 0.4912420382165605
Check guardrails
Precentage of valid sites: 0.6759956942949408
Minor allele frequency: 0.4912420382165605
Passed guardrails
Slice: 2/3. Site: 7. Done with individual 0/929
Slice: 2/3. Site: 7. Done with individual 100/929
Slice: 2/3. Site: 7. Done with individual 200/929
Slice: 2/3. Site: 7. Done with individual 300/929
Slice: 2/3. Site: 7. Done with individual 400/929
Slice: 2/3. Site: 7. Done with individual 500/929
Slice: 2/3. 

Slice: 2/3. Site: 17. Done with individual 300/929
Slice: 2/3. Site: 17. Done with individual 400/929
Slice: 2/3. Site: 17. Done with individual 500/929
Slice: 2/3. Site: 17. Done with individual 600/929
Slice: 2/3. Site: 17. Done with individual 700/929
Slice: 2/3. Site: 17. Done with individual 800/929
Slice: 2/3. Site: 17. Done with individual 900/929
Site index: 18, non ref allele frequency: 0.5068762278978389
Site index: 18, ref allele frequency: 0.4931237721021611
Check guardrails
Precentage of valid sites: 0.5479009687836384
Minor allele frequency: 0.4931237721021611
Passed guardrails
Slice: 2/3. Site: 18. Done with individual 0/929
Slice: 2/3. Site: 18. Done with individual 100/929
Slice: 2/3. Site: 18. Done with individual 200/929
Slice: 2/3. Site: 18. Done with individual 300/929
Slice: 2/3. Site: 18. Done with individual 400/929
Slice: 2/3. Site: 18. Done with individual 500/929
Slice: 2/3. Site: 18. Done with individual 600/929
Slice: 2/3. Site: 18. Done with individual 700

KeyboardInterrupt: 

## Merge outputs

In [11]:
#merge dist files
def merge_slices_dist_counts_files(files):
    # get number of individuals from the first file:
    with open(files[0],'r') as f:
        l = f.readline()
        n = len(l.split(' '))+1
    # init
    pairwise_counts = _build_pairwise_db(n, 0)
    pairwise_dist = _build_pairwise_db(n, 0.0)

    # go over the files and sum the values
    for f in files:
        with open(f,'r') as f:
            # note that while i can represent the index of the individual, j doesnot - it is merly the entrance in the matrix.
            i=-1
            for line in f.readlines():
                i+=1
                # drop the \n
                cs_ds = line[:-1].split(' ')
                j=-1
                # go over the counts(cs) and distances(ds)
                for c_d in cs_ds:
                    j+=1
                    c,d = c_d.split(',')
                    pairwise_counts[i][j]+=int(c)
                    pairwise_dist[i][j]+=float(d)
    return pairwise_counts, pairwise_dist

In [15]:
p1 = r"C:\Data\HUJI\hgdp\mock\chr9\count_dist_slice_7.tsv"
p2 = r"C:\Data\HUJI\hgdp\mock\chr9\count_dist_slice_8.tsv"
files = [p1, p2]
pairwise_counts, pairwise_dist = merge_slices_dist_counts_files(files)
output_merged_count_dist_file = r"C:\Data\HUJI\hgdp\mock\chr9\count_dist_merged.tsv"
_write_pairwise_distances(output_merged_count_dist_file, pairwise_counts, pairwise_dist)


# Adhoc playground

## Compare vcf file sizes (cluster vs source ftp server)

In [4]:
sizes_cluster = "C:\Data\HUJI\hgdp\sizes_cluster.tsv"
sizes_source = "C:\Data\HUJI\hgdp\sizes_source.tsv"
import pandas as pd
source = pd.read_csv(sizes_source, sep='\t', header=None, names =  ['key','size2'])
source['size2'] = source['size2'].str.replace(' KB', '')
source.head()
cluster = pd.read_csv(sizes_cluster, sep=' ', header=None, names =  ['size1','key'])

cluster.head()
joined = cluster.join(source.set_index('key'), on='key')
joined = joined[joined['key'].str.endswith('.gz')]
joined[joined['size1']==joined['size2']]

,size1,key,size2
